# Machine learning - Features extraction

Demo to create a feature vector for protein fold classification. 
In this demo we try to classify a protein chain as either an all alpha or all beta protein based on protein sequence. We use n-grams and a Word2Vec representation of the protein sequence as a feature vector.

[Word2Vec model](https://spark.apache.org/docs/latest/mllib-feature-extraction.html#word2vec)

[Word2Vec example](https://spark.apache.org/docs/latest/ml-features.html#word2vec)

## Imports

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from mmtfPyspark.io import mmtfReader
from mmtfPyspark.webfilters import Pisces
from mmtfPyspark.filters import ContainsLProteinChain
from mmtfPyspark.mappers import StructureToPolymerChains
from mmtfPyspark.datasets import secondaryStructureExtractor
from mmtfPyspark.ml import ProteinSequenceEncoder

## Configure Spark Context

In [2]:
spark = SparkSession.builder.master("local[4]").appName("1-Features").getOrCreate()

## Read MMTF File and get a set of L-protein chains

In [3]:
pdb = mmtfReader.read_sequence_file('../resources/mmtf_reduced_sample/') \
                .flatMap(StructureToPolymerChains()) \
                .filter(ContainsLProteinChain())

## Get secondary structure content

In [14]:
data = secondaryStructureExtractor.get_dataset(pdb)

In [15]:
data.show(5)

+----------------+--------------------+----------+----------+----------+--------------------+--------------------+
|structureChainId|            sequence|     alpha|      beta|      coil|          dsspQ8Code|          dsspQ3Code|
+----------------+--------------------+----------+----------+----------+--------------------+--------------------+
|          4WMY.A|TDWSHPQFEKSTDEANT...|0.19081272|0.26855123|0.54063606|XXXXXXXXXXXXXXXXX...|XXXXXXXXXXXXXXXXX...|
|          4WMY.B|TDWSHPQFEKSTDEANT...|0.17081851|0.26334518| 0.5658363|XXXXXXXXXXXXXXXXX...|XXXXXXXXXXXXXXXXX...|
|          4WN5.A|GSHMGRGAFLSRHSLDM...| 0.2962963|0.37962964|0.32407406|XXCCCCCCEEEEECTTC...|XXCCCCCCEEEEECCCC...|
|          4WN5.B|GSHMGRGAFLSRHSLDM...|0.33333334|0.37142858| 0.2952381|XXXXXCCCEEEEECTTC...|XXXXXCCCEEEEECCCC...|
|          4WND.A|GPGSMEASCLELALEGE...| 0.8358663|       0.0|0.16413374|XXXXCCSCHHHHHHHHH...|XXXXCCCCHHHHHHHHH...|
+----------------+--------------------+----------+----------+----------+--------

## Define add_protein_fold_type function

In [16]:
def add_protein_fold_type(data, minThreshold, maxThreshold):
    '''
    Adds a column "foldType" with three major secondary structure class:
    "alpha", "beta", "alpha+beta", and "other" based upon the fraction of alpha/beta content.

    The simplified syntax used in this method relies on two imports:
        from pyspark.sql.functions import when
        from pyspark.sql.functions import col

    Attributes:
        data (Dataset<Row>): input dataset with alpha, beta composition
        minThreshold (float): below this threshold, the secondary structure is ignored
        maxThreshold (float): above this threshold, the secondary structure is ignored
    '''

    return data.withColumn("foldType", \
                           when((col("alpha") > maxThreshold) & (col("beta") < minThreshold), "alpha"). \
                           when((col("beta") > maxThreshold) & (col("alpha") < minThreshold), "beta"). \
                           when((col("alpha") > maxThreshold) & (col("beta") > maxThreshold), "alpha+beta"). \
                           otherwise("other")\
                           )

## Classify chains by secondary structure type

In [17]:
data = add_protein_fold_type(data, minThreshold=0.05, maxThreshold=0.15)

In [18]:
data.show()

+----------------+--------------------+-----------+-----------+----------+--------------------+--------------------+----------+
|structureChainId|            sequence|      alpha|       beta|      coil|          dsspQ8Code|          dsspQ3Code|  foldType|
+----------------+--------------------+-----------+-----------+----------+--------------------+--------------------+----------+
|          4WMY.A|TDWSHPQFEKSTDEANT...| 0.19081272| 0.26855123|0.54063606|XXXXXXXXXXXXXXXXX...|XXXXXXXXXXXXXXXXX...|alpha+beta|
|          4WMY.B|TDWSHPQFEKSTDEANT...| 0.17081851| 0.26334518| 0.5658363|XXXXXXXXXXXXXXXXX...|XXXXXXXXXXXXXXXXX...|alpha+beta|
|          4WN5.A|GSHMGRGAFLSRHSLDM...|  0.2962963| 0.37962964|0.32407406|XXCCCCCCEEEEECTTC...|XXCCCCCCEEEEECCCC...|alpha+beta|
|          4WN5.B|GSHMGRGAFLSRHSLDM...| 0.33333334| 0.37142858| 0.2952381|XXXXXCCCEEEEECTTC...|XXXXXCCCEEEEECCCC...|alpha+beta|
|          4WND.A|GPGSMEASCLELALEGE...|  0.8358663|        0.0|0.16413374|XXXXCCSCHHHHHHHHH...|XXXXCCCCH

## Create a Word2Vec representation of the protein sequences

**n = 2**     # create 2-grams 

**windowSize = 25**    # 25-amino residue window size for Word2Vector

**vectorSize = 50**    # dimension of feature vector

In [8]:
encoder = ProteinSequenceEncoder(data)
data = encoder.overlapping_ngram_word2vec_encode(n=2, windowSize=25, vectorSize=50).cache()

data.toPandas().head(5)

,structureChainId,sequence,alpha,beta,coil,dsspQ8Code,dsspQ3Code,foldType,ngram,features
0,3F5B.A,SNAMMIKASTNEFRFCFKQMNKSQHELVLGWIHQPHINEWLHGDGL...,0.372093,0.319767,0.308140,XCTTCXXXXXXCCCEEEEECCGGGHHHHHHHTTSHHHHTTSCHHHH...,XCCCCXXXXXXCCCEEEEECCHHHHHHHHHHCCCHHHHCCCCHHHH...,alpha+beta,"[SN, NA, AM, MM, MI, IK, KA, AS, ST, TN, NE, E...","[-0.07528987704925445, 0.07085813883465628, -0..."
1,3F5O.A,MTSMTQSLREVIKAMTKARNFERVLGKITLVSAAPGKVICEMKVEE...,0.275362,0.413043,0.311594,XCHHHHHHHHHHHHHTTSSSGGGGGTTCEEEEEETTEEEEEEECCG...,XCHHHHHHHHHHHHHCCCCCHHHHHCCCEEEEEECCEEEEEEECCH...,alpha+beta,"[MT, TS, SM, MT, TQ, QS, SL, LR, RE, EV, VI, I...","[-0.1981162717660927, -0.3282648298990767, 0.1..."
2,3F5O.B,MTSMTQSLREVIKAMTKARNFERVLGKITLVSAAPGKVICEMKVEE...,0.264706,0.426471,0.308824,XXXCHHHHHHHHHHHHTCSSGGGGCCSEEEEEEETTEEEEEEECCG...,XXXCHHHHHHHHHHHHCCCCHHHHCCCEEEEEEECCEEEEEEECCH...,alpha+beta,"[MT, TS, SM, MT, TQ, QS, SL, LR, RE, EV, VI, I...","[-0.1981162717660927, -0.3282648298990767, 0.1..."
3,3F5O.C,MTSMTQSLREVIKAMTKARNFERVLGKITLVSAAPGKVICEMKVEE...,0.284672,0.416058,0.299270,XXCHHHHHHHHHHHHHHCSSGGGGGTTCEEEEEETTEEEEEEECCG...,XXCHHHHHHHHHHHHHHCCCHHHHHCCCEEEEEECCEEEEEEECCH...,alpha+beta,"[MT, TS, SM, MT, TQ, QS, SL, LR, RE, EV, VI, I...","[-0.1981162717660927, -0.3282648298990767, 0.1..."
4,3F5O.D,MTSMTQSLREVIKAMTKARNFERVLGKITLVSAAPGKVICEMKVEE...,0.277372,0.416058,0.306569,XXCHHHHHHHHHHHHHTCSSGGGGGTTCEEEEEETTEEEEEEECCG...,XXCHHHHHHHHHHHHHCCCCHHHHHCCCEEEEEECCEEEEEEECCH...,alpha+beta,"[MT, TS, SM, MT, TQ, QS, SL, LR, RE, EV, VI, I...","[-0.1981162717660927, -0.3282648298990767, 0.1..."


## Keep only a subset of relevant fields for further processing

In [9]:
data = data.select(['structureChainId','alpha','beta','coil','foldType','features'])

## Write to parquet file

In [10]:
data.write.mode('overwrite').format('parquet').save('./input_features')

## Terminate Spark

In [11]:
sc.stop()